<h2> An implementation of Associative Neural Network using Discrete HopField Net</h2>

In [39]:
import numpy as np
import copy

In [ ]:
input_list = [[1,1,-1,-1,-1,1],[1,-1,-1,1,-1,-1],[-1,-1,1,1,1,-1],[-1,1,1,-1,1,1]]

In [250]:
#s = np.array(([1,1,-1,-1,-1,1],[1,-1,-1,1,-1,-1],[-1,-1,1,1,1,-1],[-1,1,1,-1,1,1]))
s = np.asarray(input_list)
s

array([[ 1,  1, -1, -1, -1,  1],
       [ 1, -1, -1,  1, -1, -1],
       [-1, -1,  1,  1,  1, -1],
       [-1,  1,  1, -1,  1,  1]])

In [251]:
 t = s.T

In [252]:
print('Shapes- Input {}, Output {}'.format(s.shape,t.shape))

Shapes- Input (4, 6), Output (6, 4)


In [253]:
dimensions = [s.shape[1],t.shape[0]]
print(dimensions)

[6, 6]


In [402]:
class NeuralNetwork:
    def __init__(self,dimensions):
        '''
        Initializing the weight vectors
        '''
        self.parameters = {}
        self.basin_state_dict = {}
        self.parameters['W'] = np.zeros([dimensions[0],dimensions[1]])
        print('Weight Matrix defined as {}'.format(self.parameters["W"].shape))
    
    def compute_weight(self,X,t):
        '''
        Weight is calculated using the Hebb's rule
        '''
        dw = np.dot(t,X)
        np.fill_diagonal(dw,0)
        dw = dw.astype(int)
        return dw
    
    def train(self,S,t):
        dw = self.compute_weight(S,t)
        self.parameters['W'] = self.parameters['W']+dw
    
    def compute_sum(self,order,y):
        W = self.parameters['W']
        total = 0;
        for i in range(0,len(y)):
            #print('Weight {}'.format(W[i][order]))
            total = total+(y[i]*W[i][order])
        
        return total;
                           
    def activate(self,yin,order,y):
        theta = 0;
        if yin > theta:
            yi = 1
        elif yin < theta:
            yi = -1
        elif yin == 0:
            y = y[order]
        
        y[order] = yi
        #print('post activate',y)
        return y
    
    def _check_basin_state(self,x,y,index):
        basin_map = self.basin_state_dict
        y_copy = copy.copy(y)
        if x not in input_list:
            #print(x,y)
            if index in basin_map:
                basin_map[index].append(y_copy)
            else:
                basin_list=[]
                basin_list.append(y_copy)
                basin_map[index]=basin_list
        
        self.basin_state_dict = basin_map
        
    def compute_net_input(self,order,x,y,basin_index):
        #print('pre activate',y)
        total = self.compute_sum(order,y)
        yin = x[order]+total
        #print("order {},total {},x[order]: {},y[order]: {},yin: {},y-- {}".format(order,total,x[order],y[order],yin,y))
        y = self.activate(yin,order,y)
        #print("order {},total {},x[order]: {},y[order]: {},yin: {},y-- {}".format(order,total,x[order],y[order],yin,y))
        self._check_basin_state(x,y,basin_index)
        
        return y
                           
        
    def get_association(self,X,read_order):
        index = 0;
        assocations = []
        while index < len(X):
            x = X[index]
            #x = [-1,-1,1,1,1,1]
            y = copy.copy(x)
            print("Before Index->{}.....x: {}, y: {}".format(index,x,y)) 
            repeat_counter = 0
            while repeat_counter <= 1:
                for i in range(len(x)):
                    #print("x: {}, y: {}".format(x,y))
                    #print('index',i)
                    y = self.compute_net_input(read_order[i],x,y,index)
                if y != x:
                    #print(y,x,i,repeat_counter)
                    repeat_counter = repeat_counter+1
                else:
                    repeat_counter = 2
                     
        
            print("After  Index->{}.....x: {}, y: {}".format(index,x,y))                             
            assocations.append(y)
            index = index+1
        return assocations

In [394]:
def combinationsByKFlip(vector,start,k,end,combination_list,result_list):
    '''
    Create bipolar input vector set
    '''
    if vector not in combination_list:
        combination_list.insert(start,vector)
        
    
    if k == 0:
        result_list.append(vector)
        return result_list
    
    for i in range(start,end):
        vector = _flipElement(vector,i)
        combinationsByKFlip(vector,i+1,k-1,end,combination_list,result_list)
        vector =  combination_list[start]
   
    return result_list


def _flipElement(vector,i):
    vectorCopy = copy.copy(vector)
    if vector[i] == 1:
        vectorCopy[i] = -1
    elif vector[i] == -1:
        vectorCopy[i] = 1
    elif vector[i] == 0:
        vectorCopy[i] = 0
    return vectorCopy

In [395]:
input_vector = [1,1,-1,-1,-1,1]
bipolar_input = []
for i in range(0,len(input_vector)+1):
    start = 0
    end = len(input_vector)
    combination_list = []
    result_list = []
    result = combinationsByKFlip(input_vector,start,i,end,combination_list,result_list)
    bipolar_input.extend(result)


<b>Helper Functions to create list distinctions and validations</b>

In [396]:
#Validate if all the training data are present

def validate_list(input_list,generated_list):
    #print(generated_list)
    for x in map(lambda u:u,filter(lambda x:x in input_list,generated_list)):
        print (x)


def retreive_collections(input_list, association_list, splitby=None):
    '''
    function to split based on equiibrium or order or basin
    '''
    equilibrium = []
    spurious = []  
    if splitby == 'equilibrium':
        for x in association_list:
            if x in input_list:
                #print(x)
                equilibrium.append(x)
            else:
                spurious.append(x)
    else:
        pass
    
    return equilibrium,spurious
    

In [288]:
validate_list(input_list,bipolar_input)

[1, 1, -1, -1, -1, 1]
[-1, 1, 1, -1, 1, 1]
[1, -1, -1, 1, -1, -1]
[-1, -1, 1, 1, 1, -1]


In [279]:
f"Total bipolar input combinations..{len(bipolar_input)}"

'Total bipolar input combinations..64'

In [403]:
hopNet = NeuralNetwork(dimensions)
hopNet.train(s,t)
print('Weight Matrix \n{}'.format(hopNet.parameters['W']))

Weight Matrix defined as (6, 6)
Weight Matrix 
[[ 0.  0. -4.  0. -4.  0.]
 [ 0.  0.  0. -4.  0.  4.]
 [-4.  0.  0.  0.  4.  0.]
 [ 0. -4.  0.  0.  0. -4.]
 [-4.  0.  4.  0.  0.  0.]
 [ 0.  4.  0. -4.  0.  0.]]


In [404]:
read_order = [4,5,0,1,2,3]
associations_training_samples = hopNet.get_association(input_list,read_order)

Before Index->0.....x: [1, 1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
After  Index->0.....x: [1, 1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->1.....x: [1, -1, -1, 1, -1, -1], y: [1, -1, -1, 1, -1, -1]
After  Index->1.....x: [1, -1, -1, 1, -1, -1], y: [1, -1, -1, 1, -1, -1]
Before Index->2.....x: [-1, -1, 1, 1, 1, -1], y: [-1, -1, 1, 1, 1, -1]
After  Index->2.....x: [-1, -1, 1, 1, 1, -1], y: [-1, -1, 1, 1, 1, -1]
Before Index->3.....x: [-1, 1, 1, -1, 1, 1], y: [-1, 1, 1, -1, 1, 1]
After  Index->3.....x: [-1, 1, 1, -1, 1, 1], y: [-1, 1, 1, -1, 1, 1]


In [333]:
validate_list(input_list,associations_training_samples)

[1, 1, -1, -1, -1, 1]
[1, -1, -1, 1, -1, -1]
[-1, -1, 1, 1, 1, -1]
[-1, 1, 1, -1, 1, 1]


In [405]:
associations_total_samples = hopNet.get_association(bipolar_input,read_order)

Before Index->0.....x: [1, 1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
After  Index->0.....x: [1, 1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->1.....x: [-1, 1, -1, -1, -1, 1], y: [-1, 1, -1, -1, -1, 1]
After  Index->1.....x: [-1, 1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->2.....x: [1, -1, -1, -1, -1, 1], y: [1, -1, -1, -1, -1, 1]
After  Index->2.....x: [1, -1, -1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->3.....x: [1, 1, 1, -1, -1, 1], y: [1, 1, 1, -1, -1, 1]
After  Index->3.....x: [1, 1, 1, -1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->4.....x: [1, 1, -1, 1, -1, 1], y: [1, 1, -1, 1, -1, 1]
After  Index->4.....x: [1, 1, -1, 1, -1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->5.....x: [1, 1, -1, -1, 1, 1], y: [1, 1, -1, -1, 1, 1]
After  Index->5.....x: [1, 1, -1, -1, 1, 1], y: [1, 1, -1, -1, -1, 1]
Before Index->6.....x: [1, 1, -1, -1, -1, -1], y: [1, 1, -1, -1, -1, -1]
After  Index->6.....x: [1, 1, -1, -1, -1, -1], y: [1, 1, -1, -1, -1, 1]
Before

In [399]:
#test_list = [[-1, 1, 1, 1, 1, -1],[-1, -1, 1, 1, 1, -1]]
equilbrium,spurious = retreive_collections(input_list,associations_total_samples,splitby='equilibrium')


In [400]:
print('Total number of patterns in the equilbrium state {}'.format(len(equilbrium)))
print('Total number of patterns in the spurious state {}'.format(len(spurious)))

Total number of patterns in the equilbrium state 64
Total number of patterns in the spurious state 0


In [406]:
basin_map = hopNet.basin_state_dict

In [407]:
for index in basin_map:
    print(index)
    print(bipolar_input[index])
    print(basin_map[index])

1
[-1, 1, -1, -1, -1, 1]
[[-1, 1, -1, -1, -1, 1], [-1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1]]
2
[1, -1, -1, -1, -1, 1]
[[1, -1, -1, -1, -1, 1], [1, -1, -1, -1, -1, 1], [1, -1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1]]
3
[1, 1, 1, -1, -1, 1]
[[1, 1, 1, -1, -1, 1], [1, 1, 1, -1, -1, 1], [1, 1, 1, -1, -1, 1], [1, 1, 1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1]]
4
[1, 1, -1, 1, -1, 1]
[[1, 1, -1, 1, -1, 1], [1, 1, -1, 1, -1, 1], [1, 1, -1, 1, -1, 1], [1, 1

In [425]:
convergence_map = {}
for index in basin_map:
    convergence_index = 0;
    for state in basin_map[index]:
        if state not in input_list:
            convergence_index = convergence_index+1
        else:
            convergence_map[index] = (convergence_index,state,bipolar_input[index],input_list.index(state))
            break

In [426]:
print(convergence_map[2])

(3, [1, 1, -1, -1, -1, 1], [1, -1, -1, -1, -1, 1], 0)


In [434]:
convergence_freq = {}
for key in convergence_map:
    val = convergence_map[key]
    #print(val[0],val[3],val[1])
    field = val[3]
    if field not in convergence_freq:
        count = 1
        convergence_freq[field] = count
    else:
        convergence_freq[field] = convergence_freq[field]+1
        #print(field,convergence_freq[field])

In [435]:
convergence_freq

{0: 15, 1: 15, 2: 15, 3: 15}